In [1]:
!pip3 install -q transformers
!pip3 install -q datasets
!pip3 install emoji==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=6dd4b6bc8a6cc542c31f71b2d7726be7fe3133a4fdab708b1bfa115d13323ce8
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/3

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

folder_path = '/content/drive/My Drive/Colab Notebooks/CS 6730 Project' # Change the path to the folder where the assignment is stored in Google Drive.

os.listdir(folder_path)

os.chdir(folder_path)

print('Current working directory -', os.getcwd())

Current working directory - /content/drive/My Drive/Colab Notebooks/CS 6730 Project


In [5]:
import pandas as pd
df = pd.read_csv('office_lines.csv')
print(df.head())

   speaker                                               line
0  Michael  All right Jim. Your quarterlies look very good...
1      Jim         Oh, I told you. I couldn't close it. So...
2  Michael  So you've come to the master for guidance? Is ...
3      Jim         Actually, you called me in here, but yeah.
4  Michael    All right. Well, let me show you how it's done.


In [6]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', top_k=None)

In [ ]:
def classify_emotions(text):
  predictions = classifier(text, top_k=None)
  return {item['label']: item['score'] for item in predictions}
df['emotion_scores'] = df['line'].apply(classify_emotions)
emotion_df = pd.json_normalize(df['emotion_scores'])
df = pd.concat([df, emotion_df], axis=1)

In [ ]:
# List of the 16 selected characters
selected_characters = [
    "Michael", "Dwight", "Jim", "Pam", "Angela", "Kevin", "Oscar",
    "Toby", "Creed", "Ryan", "Kelly", "Andy", "Stanley",
    "Phyllis", "Darryl", "Erin"
]

# Filter the DataFrame to include only the selected characters
df_filtered = df[df['speaker'].isin(selected_characters)]

In [ ]:
grouped_df = df_filtered.groupby('speaker')[['joy', 'sadness', 'anger', 'love', 'fear', 'surprise']].mean()
print(grouped_df)

In [ ]:
import matplotlib.pyplot as plt
def plot_donut_chart(speaker, emotion_scores):
  labels = emotion_scores.index
  values = emotion_scores.values
  fig, ax = plt.subplots(figsize=(6, 6))
  colors = ['#FFD700', '#1E90FF', '#FF6347', '#32CD32', '#FF69B4', '#9370DB']
  wedges, texts, autotexts = ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=140, pctdistance=0.85, colors=colors,)
  center_circle = plt.Circle((0, 0), 0.70, fc='white')
  fig.gca().add_artist(center_circle)
  plt.title(f"Emotion Distribution for {speaker}", fontsize=14)
  plt.savefig(f"{speaker}_donut_chart.png", bbox_inches='tight')
  plt.show()
for speaker in grouped_df.index:
  emotion_scores = grouped_df.loc[speaker]
  plot_donut_chart(speaker, emotion_scores)